In [ ]:
- date: 2021-09-17 10:17:17
- author: Jerry Su
- slug: Faiss
- title: Faiss
- category: 
- tags: Machine Learning, Faiss

1.faiss add是增量增加吗？

2.faiss index训练是在每次add新数据，都需要重新训练吗？

[https://github.com/facebookresearch/faiss/wiki](https://github.com/facebookresearch/faiss/wiki)

[https://github.com/facebookresearch/faiss](https://github.com/facebookresearch/faiss)

[推荐召回算法-6如何从十亿数据找出你喜欢的内容-向量检索技术](https://www.bilibili.com/video/BV15v4118753?from=search&seid=9527663759693988199&spm_id_from=333.337.0.0)

[使用 Faiss 进行海量特征的相似度匹配](https://bbs.cvmart.net/articles/3887)

In [3]:
import faiss
import numpy as np

### 生成随机数据

`Faiss`处理固定维数`d`的向量集合，通常为 10 到 100 秒。 这些集合可以存储在矩阵中。
我们假设行优先存储，即。 向量编号 i 的第 j 个分量存储在矩阵的第 i 行、第 j 列中。 `Faiss`仅使用`32`位浮点矩阵。

- `xb`用于数据库，它包含所有必须编入索引的向量，我们将在其中进行搜索。大小`nb * d`

- `xq`用于查询向量，我们需要找到最近的邻居。大小`nq * d`，单个查询向量`nq = 1`

In [14]:
d = 64              # 向量维度
nb = 1000           # 数据库大小
nq = 10             # 待查询query大小
np.random.seed(42)  # 可复现

xb = np.random.random((nb, d)).astype('float32')  # 均匀分布，数据类型必须float32
xb[:, 0] += np.arange(nb) / 1000                  # 给向量的第0维（即，索引维度）增加一个小的偏移量

xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000

In [11]:
print(xb.shape, xq.shape)

(1000, 64) (10, 64)


### 创建索引并添加数据

- `Faiss`是围绕`Index`对象构建的。 它封装了一组数据库向量，并可选择对它们进行预处理以提高搜索效率。 
索引有很多种，我们将使用最简单的版本，只对它们执行暴力`L2`距离搜索：`IndexFlatL2`。

- 所有索引在创建时都需要知道，操作向量的维数，即d。然后，大多数索引还需要一个训练阶段，以分析向量的分布。
对于全量暴力搜索`IndexFlatL2`，可以跳过这个操作。

- `Index`创建和训练后，对其都可以执行两个操作：`add`和`search`。